In [8]:
import strym
from strym import strymread
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import pymysql
import re
from strym import strymmap

In [2]:
can_file= "2021-04-06-15-13-00_2T3Y1RFV8KC014025_CAN_Messages.csv"
r =strymread(csvfile=can_file)
state_var = r.state_space(rate = 10, cont_method='nearest')
state_var

[2022_03_05_20_20_36] (root) INFO: Vehicle model infered is toyota-rav4-2019


2022-03-05 20:21:01.442152: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-03-05 20:21:01.507381: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


,Time,speed,distance_covered,accelx,accely,accelz,steer_torque,yaw_rate,steer_rate,steer_angle,steer_fraction,wheel_speed_fl,wheel_speed_fr,wheel_speed_rl,wheel_speed_rr,lead_distance,acc_status,relative_vel
Clock,,,,,,,,,,,,,,,,,,
2021-04-06 15:13:01.273171968,1.617722e+09,0.00,0.000000,0.000,-4.0,46.0,-7.0,0.00,0.0,-12.0,0.4,0.00,0.00,0.00,0.00,252.0,0.0,0.0
2021-04-06 15:13:01.373192192,1.617722e+09,0.00,0.000000,0.000,-4.0,46.0,-7.0,0.00,0.0,-12.0,0.4,0.00,0.00,0.00,0.00,252.0,0.0,0.0
2021-04-06 15:13:01.473212160,1.617722e+09,0.00,0.000000,0.000,-4.0,46.0,-7.0,0.00,0.0,-12.0,0.4,0.00,0.00,0.00,0.00,252.0,0.0,0.0
2021-04-06 15:13:01.573232384,1.617722e+09,0.00,0.000000,0.000,-4.0,46.0,-7.0,0.00,0.0,-12.0,0.4,0.00,0.00,0.00,0.00,252.0,0.0,0.0
2021-04-06 15:13:01.673252608,1.617722e+09,0.00,0.000000,0.000,-4.0,46.0,-7.0,0.00,0.0,-12.0,0.4,0.00,0.00,0.00,0.00,252.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-04-06 15:23:39.301722368,1.617723e+09,6.24,32468.011328,-0.779,12.0,159.0,-32.0,14.00,4.0,321.0,0.4,5.59,6.83,5.38,6.78,252.0,5.0,0.0
2021-04-06 15:23:39.401742336,1.617723e+09,5.91,32468.617797,-0.704,-8.0,235.0,-40.0,14.50,2.0,321.0,0.4,5.25,6.68,4.97,6.62,252.0,5.0,0.0
2021-04-06 15:23:39.501762560,1.617723e+09,5.85,32469.203647,-0.640,-10.0,43.0,-46.0,12.00,1.0,321.0,0.4,5.38,6.37,4.20,5.28,252.0,5.0,0.0


In [3]:
pd.set_option('precision',10)

In [4]:
df_fact_drive_data = state_var[['Time','speed','accelx','relative_vel','lead_distance','acc_status']]
df_fact_drive_data.rename(columns={'Time':'elapsed_time','accelx':'acceleration'},inplace=True)
df_fact_drive_data
# df_fact_drive_data.round({'Time':10})

/Users/zhangjunqing/opt/anaconda3/envs/stream/lib/python3.7/site-packages/pandas/core/frame.py:5047: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,elapsed_time,speed,acceleration,relative_vel,lead_distance,acc_status
Clock,,,,,,
2021-04-06 15:13:01.273171968,1.6177219813e+09,0.00,0.000,0.0,252.0,0.0
2021-04-06 15:13:01.373192192,1.6177219814e+09,0.00,0.000,0.0,252.0,0.0
2021-04-06 15:13:01.473212160,1.6177219815e+09,0.00,0.000,0.0,252.0,0.0
2021-04-06 15:13:01.573232384,1.6177219816e+09,0.00,0.000,0.0,252.0,0.0
2021-04-06 15:13:01.673252608,1.6177219817e+09,0.00,0.000,0.0,252.0,0.0
...,...,...,...,...,...,...
2021-04-06 15:23:39.301722368,1.6177226193e+09,6.24,-0.779,0.0,252.0,5.0
2021-04-06 15:23:39.401742336,1.6177226194e+09,5.91,-0.704,0.0,252.0,5.0
2021-04-06 15:23:39.501762560,1.6177226195e+09,5.85,-0.640,0.0,252.0,5.0


In [29]:
df_fact_drive_data = df_fact_drive_data.round({'elapsed_time':1})
df_fact_drive_data

,elapsed_time,speed,acceleration,relative_vel,lead_distance,acc_status
Clock,,,,,,
2021-04-06 15:13:01.273171968,1.6177219813e+09,0.00,0.000,0.0,252.0,0.0
2021-04-06 15:13:01.373192192,1.6177219814e+09,0.00,0.000,0.0,252.0,0.0
2021-04-06 15:13:01.473212160,1.6177219815e+09,0.00,0.000,0.0,252.0,0.0
2021-04-06 15:13:01.573232384,1.6177219816e+09,0.00,0.000,0.0,252.0,0.0
2021-04-06 15:13:01.673252608,1.6177219817e+09,0.00,0.000,0.0,252.0,0.0
...,...,...,...,...,...,...
2021-04-06 15:23:39.301722368,1.6177226193e+09,6.24,-0.779,0.0,252.0,5.0
2021-04-06 15:23:39.401742336,1.6177226194e+09,5.91,-0.704,0.0,252.0,5.0
2021-04-06 15:23:39.501762560,1.6177226195e+09,5.85,-0.640,0.0,252.0,5.0


In [18]:
file= "2021-04-06-15-13-00_2T3Y1RFV8KC014025_GPS_Messages.csv"
g = strymmap(csvfile=file)
g.dataframe = g.dataframe.dropna()

[2022_03_05_20_34_16] (root) INFO: Reading GPS file 2021-04-06-15-13-00_2T3Y1RFV8KC014025_GPS_Messages.csv
GPS signal first acquired at 2021-04-06 11:10:04:300000


In [46]:
gps_dataframe = g.dataframe[['Long','Lat','Alt','Systime']]

In [47]:
def get_start_end_time(time_series):
    return time_series[3],time_series[-1]
start_time,end_time=get_start_end_time(gps_dataframe['Systime'])

In [48]:
gps_dataframe=gps_dataframe[gps_dataframe['Systime']>start_time]
gps_dataframe=gps_dataframe[gps_dataframe['Systime']<end_time]
gps_dataframe=gps_dataframe.round({'Systime':1})
gps_dataframe =gps_dataframe.rename(columns={'Systime':'elapsed_time','Long':'longitude','Lat':'latitude','Alt':'altitude'})
gps_dataframe

,longitude,latitude,altitude,elapsed_time
Clock,,,,
2021-04-06 18:12:39.329999872,-111.2816543579,32.5247001648,586.4000244141,1.6177219880e+09
2021-04-06 18:12:39.500000000,-111.2816619873,32.5247039795,586.5000000000,1.6177219882e+09
2021-04-06 18:12:39.600000000,-111.2816619873,32.5247077942,586.7000122070,1.6177219883e+09
2021-04-06 18:12:39.800000000,-111.2816696167,32.5247077942,586.7999877930,1.6177219885e+09
2021-04-06 18:12:39.900000000,-111.2816696167,32.5247116089,586.9000244141,1.6177219886e+09
...,...,...,...,...
2021-04-06 18:23:08.200000000,-111.2161941528,32.4629096985,602.9000244141,1.6177226169e+09
2021-04-06 18:23:08.300000000,-111.2162017822,32.4629096985,602.9000244141,1.6177226170e+09
2021-04-06 18:23:08.400000000,-111.2162017822,32.4629096985,602.9000244141,1.6177226171e+09


In [52]:
df_merge=gps_dataframe.merge(df_fact_drive_data, how='inner', on='elapsed_time')

In [53]:
df_merge

,longitude,latitude,altitude,elapsed_time,speed,acceleration,relative_vel,lead_distance,acc_status
0,-111.2816543579,32.5247001648,586.4000244141,1.6177219880e+09,0.00,0.000,0.0,252.0,0.0
1,-111.2816619873,32.5247039795,586.5000000000,1.6177219882e+09,0.00,0.000,0.0,252.0,0.0
2,-111.2816619873,32.5247077942,586.7000122070,1.6177219883e+09,0.00,0.000,0.0,252.0,0.0
3,-111.2816696167,32.5247077942,586.7999877930,1.6177219885e+09,0.00,0.000,0.0,252.0,0.0
4,-111.2816696167,32.5247116089,586.9000244141,1.6177219886e+09,0.00,0.000,0.0,252.0,0.0
...,...,...,...,...,...,...,...,...,...
5337,-111.2161941528,32.4629096985,602.9000244141,1.6177226169e+09,12.88,-0.381,0.0,252.0,5.0
5338,-111.2162017822,32.4629096985,602.9000244141,1.6177226170e+09,12.83,-0.096,0.0,252.0,5.0
5339,-111.2162017822,32.4629096985,602.9000244141,1.6177226171e+09,12.54,-0.377,0.0,252.0,5.0
5340,-111.2162094116,32.4629135132,602.9000244141,1.6177226172e+09,12.21,-0.387,0.0,252.0,5.0
